## Glass training
 This notebook loads the Glass identification dataset (https://archive.ics.uci.edu/dataset/42/glass+identification), preprocesses it and trains a simple model

In [1]:
import sys
import os
PROJ_DIR = os.path.realpath(os.path.dirname(os.path.abspath('')))
sys.path.append(os.path.join(PROJ_DIR,'src'))



In [3]:
import numpy as np

file_data = np.load(os.path.join(PROJ_DIR, 'assets', 'data', 'avila.npz'))
x_train = file_data['x_train']
x_test = file_data['x_test']
y_train = file_data['y_train']
y_test = file_data['y_test']

Load data from file

In [7]:
import torch
class RandomMasker(torch.nn.Module):
    def __init__(self, masking_value:torch.tensor):
        super().__init__()
        self._masking_value = masking_value
    def forward(self, x): # Assumes inputs are (batch_size, num_vars)
        selection_levels = torch.rand((x.shape[0], 1)) # A different selection level for each element of the batch
        selected_pixels = torch.le(torch.rand(x.shape), selection_levels) # A different selection level for each element of the batch
        return x * selected_pixels + self._masking_value * ~selected_pixels

# Zeros
masking_value = np.zeros(x_train.shape[1])
# Mean
masking_value = np.mean(x_train, axis=0)
masker = RandomMasker(torch.tensor(masking_value).float())

Train a MLP model

In [8]:
MODEL_NEURONS = 100
MODEL_EPOCHS= 2000
MODEL_LR = 1.0e-1
MODEL_LABEL_NUM = len(np.unique(y_train))

class MLP(torch.nn.Module):
    def __init__(self, n_neurons):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(x_train.shape[1], n_neurons)
        self.ac1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_neurons, MODEL_LABEL_NUM)
        self.ac2 = torch.nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.ac1(x)
        logits = self.fc2(x)
        x = self.ac2(logits)
        return x

x_train_tensor = torch.tensor(x_train).float()
y_train_tensor = torch.tensor(y_train)
x_test_tensor = torch.tensor(x_test).float()
y_test_tensor = torch.tensor(y_test)

label_onehot = torch.zeros(y_train.shape[0], MODEL_LABEL_NUM)
label_onehot.scatter_(1, y_train_tensor.unsqueeze(1), 1)
class_weights = 1.0/label_onehot.mean(axis=0)

network = MLP(MODEL_NEURONS)
loss = torch.nn.BCELoss(weight=class_weights)
optimizer = torch.optim.Adam(network.parameters(), lr=MODEL_LR)#, weight_decay=1e-3)

#MODEL_EPOCHS = 0 # Untrained
for epoch in range(MODEL_EPOCHS):
    x_train_tensor_masked = masker(torch.tensor(x_train).float()) # A different set of RandomMasks for each batch
    optimizer.zero_grad()
    
    preds = network(x_train_tensor_masked)
    label_onehot = torch.zeros(y_train.shape[0], MODEL_LABEL_NUM)
    label_onehot.scatter_(1, y_train_tensor.unsqueeze(1), 1)
    loss_value = loss(preds, label_onehot)
    loss_value.backward()        
    optimizer.step()

    train_accuracy = (preds.argmax(dim=1) == y_train_tensor).float().mean() 

    test_preds = network.forward(x_test_tensor)        
    test_accuracy = (test_preds.argmax(dim=1) == y_test_tensor).float().mean() 
    print(f'Epoch {epoch}/{MODEL_EPOCHS} - Loss: {loss_value.item()} - Train accuracy: {train_accuracy} - Test accuracy: {test_accuracy}')  
    #if test_accuracy > 0.6: # Undertrained
    #    break

test_preds = network.forward(x_test_tensor)        
test_accuracy = (test_preds.argmax(dim=1) == y_test_tensor).float().mean() 
print(test_accuracy.item())


Epoch 0/2000 - Loss: 20.00139808654785 - Train accuracy: 0.02837967313826084 - Test accuracy: 0.08063279092311859
Epoch 1/2000 - Loss: 6.436717987060547 - Train accuracy: 0.052732501178979874 - Test accuracy: 0.32099711894989014
Epoch 2/2000 - Loss: 6.159282207489014 - Train accuracy: 0.3332694172859192 - Test accuracy: 0.40575262904167175
Epoch 3/2000 - Loss: 6.1150126457214355 - Train accuracy: 0.4039309620857239 - Test accuracy: 0.39530202746391296
Epoch 4/2000 - Loss: 6.381319999694824 - Train accuracy: 0.4033557176589966 - Test accuracy: 0.3988494873046875
Epoch 5/2000 - Loss: 5.686429023742676 - Train accuracy: 0.4053691327571869 - Test accuracy: 0.39923298358917236
Epoch 6/2000 - Loss: 6.601292133331299 - Train accuracy: 0.4052732586860657 - Test accuracy: 0.3982742130756378
Epoch 7/2000 - Loss: 6.0185441970825195 - Train accuracy: 0.40239691734313965 - Test accuracy: 0.3961648941040039
Epoch 8/2000 - Loss: 5.713531494140625 - Train accuracy: 0.40172579884529114 - Test accuracy:

In [9]:
# Save model
torch.save(network.state_dict(), os.path.join(PROJ_DIR,'assets','models','avila-ood-mean-mlp.pth'))